In [1]:
import sys
sys.path.append('/usr/lib/python3/dist-packages')
import os
import pandas as pd
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import json
#import shapefile
#from shapely.geometry  import Point, LineString, Polygon, MultiPolygon

from django.conf import settings
#from sqlalchemy import create_engine
os.environ["DJANGO_SETTINGS_MODULE"] = "mysite.settings"
import django
django.setup()
from tqdm import tqdm
import glob
import numpy as np
from django.db.models import Q, Sum, Avg

from pyproj import Proj, transform

/home/tserekh/env/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
import glob
from sklearn.neighbors import KDTree

In [3]:
import random
import itertools

In [4]:
from main_map.models import PredictedSquare

In [5]:
from pyproj import Proj, transform

PROJ3857  = Proj("+init=EPSG:3857")
PROJ4326 = Proj("+init=EPSG:4326")

In [6]:
lat_min, lon_min = 55.577315, 37.082288
lat_max, lon_max = 56.001331, 38.016126


In [7]:
x_min, y_min = transform(PROJ4326, PROJ3857, lon_min, lat_min)
x_max, y_max = transform(PROJ4326, PROJ3857, lon_max, lat_max)

In [8]:
from main_map.models import OrganizationNatClass, House

In [9]:

import glob
path = '/home/tserekh/abc_data/evil/*'
glob.glob(path)

df1 = pd.read_excel('/home/tserekh/abc_data/evil/t1_20180214_15-17-39.xlsx')
df1 = df1.drop_duplicates('store_id')

df2 = pd.read_excel('/home/tserekh/abc_data/evil/t2_2018-02-09.xlsx')
df2 =df2[df2['Выручка ']>0]
df2 = df2.groupby('id аптеки', as_index=False)[['Выручка ']].mean()

df = pd.merge(
    df1[['store_id','location_type_name','opening_type_name','brand_id','pos_x','pos_y']],
    df2,
    left_on = 'store_id',
    right_on='id аптеки')

gb_features = ['brand_id','location_type_name','opening_type_name']
df_gb = df.groupby(gb_features, as_index=False)[['Выручка ']].mean()
df = pd.merge(df, df_gb, on=gb_features)
df['rate'] = df['Выручка _x']/df['Выручка _y']

In [10]:
gb_features

['brand_id', 'location_type_name', 'opening_type_name']

In [11]:
from itertools import product

In [12]:
sq_side = 500

In [13]:
features =['x','y']
X = np.arange(x_min,x_max, sq_side)
Y = np.arange(y_min,y_max, sq_side)

In [14]:
PredictedSquare.objects.all().delete()
print(len(X)*len(Y))
for coord in tqdm_notebook(product(X,Y)):
    dic = {}
    dic['x'] = coord[0]
    dic['y'] = coord[1]
    dic['revenue_pred_model1'] = 0
    pred_square = PredictedSquare(**dic)
    pred_square.save()

34944


In [15]:
from sklearn.neighbors import NearestNeighbors


In [16]:
x_y = ['x','y']

In [17]:
coords = []
for i, row in tqdm_notebook(df[['pos_x','pos_y']].iterrows()):
    try:
        coord = transform(PROJ4326, PROJ3857, row['pos_x'], row['pos_y'])
    except RuntimeError:
        coord = [None, None]
    coords.append(coord)
df['x'] = np.array(coords)[:,0]
df['y'] = np.array(coords)[:,1]
df = df[df['x'].notnull()]
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(df[x_y])

In [18]:
mean_rev = 1707108

In [19]:
objects = PredictedSquare.objects.filter(machine_features__isnull=True).order_by('x')
#objects = PredictedSquare.objects.all().order_by('x')
print(objects.count())
for object_ in tqdm_notebook(objects):
    distances, indices = nbrs.kneighbors([[object_.x, object_.y]])
    revenue = df.iloc[indices[0][0]]['rate']*mean_rev
    object_.revenue_pred_model1 = revenue
    object_.save()

34944


In [20]:
a = PredictedSquare.objects.all()[0]

In [21]:
a.__dict__

{'_state': <django.db.models.base.ModelState at 0x7f100131e4e0>,
 'id': 3197573,
 'x': 4222481.41760952,
 'y': 7496227.10095972,
 'revenue_pred_model1': 581145.5062483,
 'revenue_pred_model2': None,
 'machine_features': None}

In [22]:
transform(PROJ3857, PROJ4326,  4127981.41760952, 7488227.10095972)

(37.082288000000005, 55.645809780810524)